## Exercise 1

Given the following data:

| Tid | Refund | Marital Status | Taxable Income (K) | Cheat |
|-----|--------|----------------|--------------------|-------|
| 1   | Yes    | Single         | 125                | No    |
| 2   | No     | Married        | 100                | No    |
| 3   | No     | Single         | 70                 | No    |
| 4   | Yes    | Married        | 120                | No    |
| 5   | No     | Divorced       | 95                 | Yes   |
| 6   | No     | Married        | 60                 | No    |
| 7   | Yes    | Divorced       | 220                | No    |
| 8   | No     | Single         | 85                 | Yes   |
| 9   | No     | Married        | 75                 | No    |
| 10  | No     | Single         | 90                 | Yes   |

What is the best first split (using Gini - note, for the continuous feature, check quartile boundaries).

In [1]:
# refund - n(4/7) y - 3 pure, ms- single - 2/4 married 4 pure divorce  1/2 

### the best first split to determine cheat would be using the Marital Status variable and split on the Married Value 

## Exercise 2

Build a decision tree to fit the [federalist papers](https://www.kaggle.com/datasets/tobyanderson/federalist-papers_) data, available in the data directory (click on the link to find out more information about this data). Note that you should restrict your analysis to papers by Hamilton or Madison.  Plot your training and test scores to pick a value for ccp_alpha. What did you pick?  Run your trained classifier on the "disputed" papers.  What does your model tell you? 

In [50]:
import pandas as pd 
import sklearn 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

df = pd.read_csv("data/federalistpapers.csv")
df[df["author"].values == 'Hamilton']
df[df["author"].values == 'Madison']
hm_df = df[df["author"].values == 'Hamilton']
hm_df = hm_df.append(df[df["author"].values == 'Madison'])
hm_df = hm_df.drop(["filename"], axis = 1)
hm_df = hm_df.reset_index()
hm_df = hm_df.drop(["index"], axis = 1)
df[df["author"] == 'Madison']
hm_df.replace({"Madison": 0, "Hamilton": 1}, inplace=True)
hm_df.head()

/var/folders/yr/ck3hvtw16ks5djc79x7fgf6r0000gn/T/ipykernel_27674/2725781921.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hm_df = hm_df.append(df[df["author"].values == 'Madison'])


,author,a,all,also,an,and,any,are,as,at,...,was,were,what,when,which,who,will,with,would,your
0,1,0.213,0.083,0.000,0.083,0.343,0.056,0.111,0.093,0.065,...,0.000,0.000,0.000,0.009,0.158,0.074,0.222,0.046,0.019,0.074
1,1,0.369,0.070,0.006,0.076,0.411,0.023,0.053,0.117,0.065,...,0.000,0.012,0.012,0.012,0.147,0.029,0.094,0.129,0.270,0.000
2,1,0.305,0.047,0.007,0.068,0.386,0.047,0.102,0.108,0.088,...,0.000,0.000,0.007,0.000,0.156,0.007,0.074,0.122,0.149,0.000
3,1,0.391,0.045,0.015,0.030,0.270,0.045,0.060,0.090,0.015,...,0.000,0.000,0.000,0.045,0.165,0.045,0.135,0.150,0.210,0.000
4,1,0.327,0.096,0.000,0.086,0.356,0.014,0.086,0.072,0.115,...,0.014,0.038,0.014,0.019,0.264,0.029,0.091,0.086,0.062,0.010


In [60]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(hm_df, hm_df["author"]):
    train_set = hm_df.loc[train_index]
    test_set = hm_df.loc[test_index]

strat_train = train_set
train_y = strat_train["author"]
strat_train = strat_train.drop("author", axis=1)
strat_test = test_set.copy()
test_y = test_set["author"]
test_x = test_set.drop("author", axis=1)

In [65]:
clf = DecisionTreeClassifier(random_state= 42)
train_error = []
test_error = []
kf = KFold(n_splits=5)

# for i, (train_index, test_index) in enumerate(kf.split(strat_train)):
#     print(f"Fold {i}:")
#     print(f"  Train: index={train_index}")
#     train = strat_train.iloc[train_index]
#     kin_y = train_y.iloc[train_index]
#     clf.fit(train, kin_y)
#     train_error.append(clf.score(train, kin_y))
#     test = strat_train.iloc[test_index]
#     kest_y = test_y.iloc[test_index]
#     train_error.append((clf.score(train, kin_y)))       

Fold 0:
  Train: index=[11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51]
Fold 1:
  Train: index=[ 0  1  2  3  4  5  6  7  8  9 10 22 23 24 25 26 27 28 29 30 31 32 33 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51]


IndexError: positional indexers are out-of-bounds

## Exercise 3

Build a voting classifier for the federalist papers, using all of the non-ensemble methods you've been exposed to in this class thus far (i.e., KNN, SVM, logistic regression, naive bayes, SGDClassifier, decision tree).

1) Compare this to a RandomForest classifier.  Which works the best?
2) Compare this to a GradientBoosting classifier.  Which works the best?
3) Add the RandomForest and GradientBoosting classifiers to your voting classifier.  Does you performance improve?

## Exercise 4

When does it make sense to use a Bagging Classifier?  In the following, explore different data parameters to develop your intuition for which classifier makes sense in which situation. 

1. Gradually increase the noise in the data (using the noise parameter).  How do the different classifiers perform.  Why?


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

X, y = make_moons(n_samples=300, noise=.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)
y_train_pred_tree = tree.predict(X_train)
print("Decision Tree Accuracy (train):", accuracy_score(y_train, y_train_pred_tree))
print("Decision Tree Accuracy (test):", accuracy_score(y_test, y_pred_tree))



log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)
y_pred_log = log_reg.predict(X_test)
y_train_pred_log = log_reg.predict(X_train)
print("Logistic Regression Accuracy (train):", accuracy_score(y_train, y_train_pred_log))
print("Logistic Regression Accuracy (test):", accuracy_score(y_test, y_pred_log))

2.  Now, do the same thing in the following.  What do you notice. How do you explain your observations?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

max_samples = 100

X, y = make_moons(n_samples=300, noise=.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

bag_tree = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, random_state=42,max_samples=max_samples)
bag_tree.fit(X_train, y_train)
y_pred_bag_train = bag_tree.predict(X_train)
y_pred_bag = bag_tree.predict(X_test)
print("Bagging Decision Tree Accuracy (train):", accuracy_score(y_train, y_pred_bag_train))
print("Bagging Decision Tree Accuracy (test):", accuracy_score(y_test, y_pred_bag))


bag_log = BaggingClassifier(LogisticRegression(), n_estimators=500, random_state=42,max_samples=max_samples)
bag_log.fit(X_train, y_train)
y_pred_bag_log_train = bag_log.predict(X_train)
y_pred_bag_log = bag_log.predict(X_test)
print("Bagging Logistic Regression Accuracy (train):", accuracy_score(y_train, y_pred_bag_log_train))
print("Bagging Logistic Regression Accuracy (test):", accuracy_score(y_test, y_pred_bag_log))

3. Now go back and start increasing the `max_samples` parameter.  How do things change? 

## Exercise 5

The "wine" dataset contains data about the chemical makeup of different varieties of wine and critics scores.  Use XGBoost to build a classifier for this data.  Manually tune the hyperparameters of the XGBoost model to try to achieve better accuracy on the test set than the baseline model. Some hyperparameters to consider tweaking:
   - `learning_rate`
   - `max_depth`
   - `n_estimators`
   - `gamma`
   - `subsample`
   - `colsample_bytree`

See [the online docs](https://xgboost.readthedocs.io/en/stable/parameter.html) for more info.

After tuning, use the `plot_importance` function again to see if feature importances have changed after tuning.


1. How did hyperparameter tuning affect the model's accuracy? Which hyperparameters seemed to have the most influence?
2. Did feature importances change after tuning? If so, why might that be?

In [ ]:
# Run this if you don't have XGBoost installed
%pip install XGBoost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import plot_importance

data = load_wine()

# We'll use a data frame to make sure we get real feature names out
X = pd.DataFrame(data.data,columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = xgb.XGBClassifier(objective='multi:softprob', random_state=42)
clf.fit(X_train, y_train)

baseline_accuracy = clf.score(X_test, y_test)
print(f"Baseline Accuracy: {baseline_accuracy:.4f}")

plot_importance(clf)
plt.show()